In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing Relevant Datasets

In [4]:
# Importing data
admissions_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/hosp/admissions.csv.gz')
icu_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/icu/icustays.csv.gz')
drg_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/hosp/drgcodes.csv.gz')
diag_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz')
d_diag_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz')
patients_df = pd.read_csv('/Users/jibuthomas/Documents/mimic-iv-2.2/hosp/patients.csv.gz')

In [5]:
drg_df.head()

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,10000032,22595853,APR,283,OTHER DISORDERS OF THE LIVER,2.0,2.0
1,10000032,22595853,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
2,10000032,22841357,APR,279,HEPATIC COMA & OTHER MAJOR ACUTE LIVER DISORDERS,3.0,2.0
3,10000032,22841357,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
4,10000032,25742920,APR,283,OTHER DISORDERS OF THE LIVER,3.0,2.0


### It looks like drg type APR and HCFA gives the same information

In [6]:
# Filtering out HCFA rows
drg_df = drg_df[drg_df['drg_type'] != 'HCFA']

In [7]:
drg_df.drg_type.value_counts()

drg_type
APR    292422
Name: count, dtype: int64

In [9]:
# Figuring out the columns in the relevant datasets
def get_columns(df):
    return df.columns.tolist()

df_list = [admissions_df, icu_df, drg_df, diag_df, d_diag_df, patients_df]
df_names = ['admissions_df', 'icu_df', 'drg_df', 'diag_df', 'd_diag_df', 'patients_df']

for df, name in zip(df_list, df_names):
    print(f"Columns in {name}: {get_columns(df)}")

Columns in admissions_df: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']
Columns in icu_df: ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los']
Columns in drg_df: ['subject_id', 'hadm_id', 'drg_type', 'drg_code', 'description', 'drg_severity', 'drg_mortality']
Columns in diag_df: ['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version']
Columns in d_diag_df: ['icd_code', 'icd_version', 'long_title']
Columns in patients_df: ['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']


### Target variable "los" found in icu_df

In [11]:
# Filter out patients with acute traumatic spinal cord injury
d_diag_df.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [21]:
d_diag_df[d_diag_df['long_title'].str.contains('spinal cord injury', case=False, na=False)]

,icd_code,icd_version,long_title
10006,8052,9,Closed fracture of dorsal [thoracic] vertebra ...
10007,8053,9,Open fracture of dorsal [thoracic] vertebra wi...
10008,8054,9,Closed fracture of lumbar vertebra without men...
10009,8055,9,Open fracture of lumbar vertebra without menti...
10010,8056,9,Closed fracture of sacrum and coccyx without m...
10011,8057,9,Open fracture of sacrum and coccyx without men...
10012,8058,9,Closed fracture of unspecified vertebral colum...
10013,8059,9,Open fracture of unspecified vertebral column ...
10014,80600,9,Closed fracture of C1-C4 level with unspecifie...
10018,80604,9,Closed fracture of C1-C4 level with other spec...


### To reduce patient factor variability, the prediction model will only predict ICU lengths of stay for people who experienced a spinal cord injury at the neck (ICD diagnostic codes 806.0, and 806.00 through 806.09)

In [29]:
diag_df_filtered = diag_df[diag_df['icd_code'].str.startswith('8060')]
print(diag_df_filtered.head())
print(f"Number of patients with acute traumatic spinal cord injury at the neck: {len(diag_df_filtered)}")

        subject_id   hadm_id  seq_num icd_code  icd_version
88662     10196368  24464472        1    80604            9
124710    10275408  20562387        1    80603            9
124713    10275408  20562387        4    80608            9
124912    10275551  29306218        1    80605            9
125199    10275842  21898344        1    80600            9
Number of patients with acute traumatic spinal cord injury at the neck: 158


In [31]:
# Check for duplicates in the subject_id and hadm_id columns
duplicates = diag_df_filtered[diag_df_filtered.duplicated(subset=['subject_id', 'hadm_id'], keep=False)]

# Display the duplicates if any
print(duplicates)

         subject_id   hadm_id  seq_num icd_code  icd_version
124710     10275408  20562387        1    80603            9
124713     10275408  20562387        4    80608            9
715232     11522912  22953852        1    80604            9
715237     11522912  22953852        6    80609            9
848976     11807843  28397513        1    80600            9
848984     11807843  28397513        9    80605            9
1240968    12607579  28936337        1    80604            9
1240971    12607579  28936337        4    80609            9
1384615    12916974  23183828        1    80600            9
1384626    12916974  23183828       12    80605            9
2349228    14936398  21985899        1    80600            9
2349232    14936398  21985899        5    80605            9
2364327    14973730  29551490        2    80601            9
2364329    14973730  29551490        4    80606            9
3140279    16594084  21161575        1    80600            9
3140283    16594084  211

In [32]:
len(duplicates)

30

In [36]:
# Remove duplicates based on subject_id and hadm_id
diag_df_filtered.drop_duplicates(subset=['subject_id', 'hadm_id'], inplace=True)

/var/folders/yg/f1g020cd6zl_n34fsq8_h_2r0000gn/T/ipykernel_3615/1366145608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diag_df_filtered.drop_duplicates(subset=['subject_id', 'hadm_id'], inplace=True)


In [37]:
len(diag_df_filtered)

143

In [38]:
for df, name in zip(df_list, df_names):
    print(f"Columns in {name}: {get_columns(df)}")

Columns in admissions_df: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']
Columns in icu_df: ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los']
Columns in drg_df: ['subject_id', 'hadm_id', 'drg_type', 'drg_code', 'description', 'drg_severity', 'drg_mortality']
Columns in diag_df: ['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version']
Columns in d_diag_df: ['icd_code', 'icd_version', 'long_title']
Columns in patients_df: ['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']


In [41]:
# Merge the datasets
merged_df = diag_df_filtered.merge(admissions_df, on=['subject_id', 'hadm_id'], how='left') \
                             .merge(patients_df, on='subject_id', how='left') \
                             .merge(icu_df, on=['subject_id', 'hadm_id'], how='left') \
                             .merge(drg_df, on=['subject_id', 'hadm_id'], how='left')

In [42]:
merged_df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,admittime,dischtime,deathtime,admission_type,admit_provider_id,...,first_careunit,last_careunit,intime,outtime,los,drg_type,drg_code,description,drg_severity,drg_mortality
0,10196368,24464472,1,80604,9,2184-05-03 13:50:00,2184-05-07 18:03:00,NaN,EW EMER.,P39FGY,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2184-05-03 14:36:00,2184-05-05 21:49:53,2.301308,APR,40,SPINAL DISORDERS & INJURIES,2.0,2.0
1,10275408,20562387,1,80603,9,2157-02-13 12:55:00,2157-02-20 12:50:00,NaN,EW EMER.,P77BSD,...,NaN,NaN,NaN,NaN,NaN,APR,912,MUSCULOSKELETAL & OTHER PROCEDURES FOR MULTIPL...,3.0,1.0
2,10275551,29306218,1,80605,9,2146-05-21 03:16:00,2146-05-29 16:20:00,NaN,EW EMER.,P23Y5G,...,NaN,NaN,NaN,NaN,NaN,APR,40,SPINAL DISORDERS & INJURIES,4.0,2.0
3,10275842,21898344,1,80600,9,2170-01-14 06:18:00,2170-01-22 16:58:00,NaN,EW EMER.,P462X1,...,NaN,NaN,NaN,NaN,NaN,APR,23,SPINAL PROCEDURES,2.0,1.0
4,10348324,27926100,1,80608,9,2137-06-29 20:21:00,2137-07-06 14:50:00,NaN,EW EMER.,P93E1Z,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2137-07-02 01:42:38,2137-07-03 23:09:38,1.893750,APR,23,SPINAL PROCEDURES,2.0,1.0


In [43]:
merged_df.shape

(150, 35)

In [44]:
# Filter Duplicates
duplicates = merged_df[merged_df.duplicated(subset=['subject_id', 'hadm_id'], keep=False)]

# Check if there are any duplicates
if not duplicates.empty:
    print("Duplicates found:")
    print(duplicates)
else:
    print("No duplicates found.")

Duplicates found:
     subject_id   hadm_id  seq_num icd_code  icd_version            admittime  \
28     12324075  24304637        1    80605            9  2122-03-02 20:13:00   
29     12324075  24304637        1    80605            9  2122-03-02 20:13:00   
30     12324075  24304637        1    80605            9  2122-03-02 20:13:00   
39     13096583  20322847        1    80605            9  2167-07-30 18:01:00   
40     13096583  20322847        1    80605            9  2167-07-30 18:01:00   
60     14309442  21702831        1    80609            9  2183-07-23 01:20:00   
61     14309442  21702831        1    80609            9  2183-07-23 01:20:00   
91     16207680  24977761        1    80601            9  2154-07-17 21:25:00   
92     16207680  24977761        1    80601            9  2154-07-17 21:25:00   
93     16207680  24977761        1    80601            9  2154-07-17 21:25:00   
126    18651030  20582772        1    80608            9  2113-02-16 06:49:00   
127    186

In [45]:
duplicates

,subject_id,hadm_id,seq_num,icd_code,icd_version,admittime,dischtime,deathtime,admission_type,admit_provider_id,...,first_careunit,last_careunit,intime,outtime,los,drg_type,drg_code,description,drg_severity,drg_mortality
28,12324075,24304637,1,80605,9,2122-03-02 20:13:00,2122-03-28 16:00:00,NaN,EW EMER.,P26BSW,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2122-03-03 11:32:10,2122-03-11 18:17:09,8.281238,APR,23,SPINAL PROCEDURES,3.0,3.0
29,12324075,24304637,1,80605,9,2122-03-02 20:13:00,2122-03-28 16:00:00,NaN,EW EMER.,P26BSW,...,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2122-03-12 03:52:25,2122-03-12 09:29:22,0.233993,APR,23,SPINAL PROCEDURES,3.0,3.0
30,12324075,24304637,1,80605,9,2122-03-02 20:13:00,2122-03-28 16:00:00,NaN,EW EMER.,P26BSW,...,Coronary Care Unit (CCU),Trauma SICU (TSICU),2122-03-12 10:13:09,2122-03-26 17:48:24,14.316146,APR,23,SPINAL PROCEDURES,3.0,3.0
39,13096583,20322847,1,80605,9,2167-07-30 18:01:00,2167-08-10 12:30:00,NaN,EW EMER.,P39FGY,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2167-08-04 10:56:29,2167-08-10 12:40:26,6.072188,APR,23,SPINAL PROCEDURES,3.0,3.0
40,13096583,20322847,1,80605,9,2167-07-30 18:01:00,2167-08-10 12:30:00,NaN,EW EMER.,P39FGY,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2167-07-30 18:22:00,2167-08-02 16:36:02,2.926412,APR,23,SPINAL PROCEDURES,3.0,3.0
60,14309442,21702831,1,80609,9,2183-07-23 01:20:00,2183-08-13 16:13:00,NaN,EW EMER.,P01LRQ,...,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2183-08-06 21:36:06,2183-08-07 16:23:10,0.782685,APR,23,SPINAL PROCEDURES,3.0,2.0
61,14309442,21702831,1,80609,9,2183-07-23 01:20:00,2183-08-13 16:13:00,NaN,EW EMER.,P01LRQ,...,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2183-07-23 02:11:00,2183-07-28 20:06:15,5.746701,APR,23,SPINAL PROCEDURES,3.0,2.0
91,16207680,24977761,1,80601,9,2154-07-17 21:25:00,2154-09-15 12:42:00,NaN,EW EMER.,P99LA7,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-08-13 16:12:08,2154-08-23 21:09:58,10.206829,APR,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCEDURE,4.0,3.0
92,16207680,24977761,1,80601,9,2154-07-17 21:25:00,2154-09-15 12:42:00,NaN,EW EMER.,P99LA7,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-07-17 21:48:00,2154-08-04 14:29:23,17.695405,APR,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCEDURE,4.0,3.0
93,16207680,24977761,1,80601,9,2154-07-17 21:25:00,2154-09-15 12:42:00,NaN,EW EMER.,P99LA7,...,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-08-05 18:28:36,2154-08-07 19:37:44,2.048009,APR,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCEDURE,4.0,3.0


In [46]:
# Remove duplicates based on subject_id and hadm_id
merged_df.drop_duplicates(subset=['subject_id', 'hadm_id'], inplace=True)  
print(f"Shape after removing duplicates: {merged_df.shape}")

Shape after removing duplicates: (143, 35)
